# 데이터셋 만드는 부분만 추려놓은 코드

## entity 앞뒤에 @, # 으로 표시, type 삽입

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [146]:
pd.set_option('display.max_colwidth', None)

### test data

In [166]:
df = pd.read_csv('../dataset/test/test_data.csv')

In [167]:
# subject_entity index 와 타입을 받아서 DataFrame 에 저장

subject_start = []
subject_end = []
subject_type = []

for i in df['subject_entity']:
    i = int(i[1:-1].split("'start_idx': ")[1].split(',')[0])

    subject_start.append(i)

for j in df['subject_entity']:
    j = int(j[1:-1].split("'end_idx': ")[1].split(',')[0])+1

    subject_end.append(j)
    
for k in df['subject_entity']:
    k = str(k[1:-1].split("'type': ")[1][1:4])

    subject_type.append(k)
    
subject_start = pd.Series(subject_start)
subject_end = pd.Series(subject_end)
subject_type = pd.Series(subject_type)

df_sub_position = pd.DataFrame({'subject_start':subject_start, 'subject_end':subject_end, 'subject_type':subject_type})

In [168]:
df_sub_position

,subject_start,subject_end,subject_type
0,7,10,ORG
1,7,14,PER
2,121,123,ORG
3,78,80,PER
4,3,6,ORG
...,...,...,...
7760,33,35,ORG
7761,133,136,ORG
7762,0,8,ORG
7763,6,13,PER


In [169]:
# subject_entity 단어 확인

sub = []

for sen, i, j in zip(df.sentence, df_sub_position.subject_start, df_sub_position.subject_end):
    sub.append(sen[i:j])

In [170]:
sub = pd.Series(sub)

In [171]:
sub

0            MBC
1        프린세스 프링
2             경찰
3             세조
4            민주당
          ...   
7760          정부
7761         종근당
7762    한국전기안전공사
7763     B. 슈나이더
7764         이승옥
Length: 7765, dtype: object

In [172]:
# object_entity index 와 타입을 받아서 DataFrame 에 저장

object_start = []
object_end = []
object_type = []

for i in df['object_entity']:
    i = int(i[1:-1].split("'start_idx': ")[1].split(',')[0])

    object_start.append(i)

for j in df['object_entity']:
    j = int(j[1:-1].split("'end_idx': ")[1].split(',')[0])+1

    object_end.append(j)
    
for k in df['object_entity']:
    k = str(k[1:-1].split("'type': ")[1][1:4])

    object_type.append(k)
    
object_start = pd.Series(object_start)
object_end = pd.Series(object_end)
object_type = pd.Series(object_type)

df_obj_position = pd.DataFrame({'object_start':object_start, 'object_end':object_end, 'object_type':object_type})

In [173]:
# subject_entity 단어 확인

obj = []

for sen, i, j in zip(df.sentence, df_obj_position.object_start, df_obj_position.object_end):
    obj.append(sen[i:j])

In [174]:
obj = pd.Series(obj)

In [175]:
obj

0       탐사기획 스트레이트
1               공주
2            1995년
3              정창손
4              권영진
           ...    
7760          이탈리아
7761            전년
7762           조성완
7763         1987년
7764            2일
Length: 7765, dtype: object

In [176]:
# subject_entity  @ {type} {entity} @ 형태로 표현 

replace_sub = []

for sen, i, j, k, word in zip(df.sentence, df_sub_position.subject_start, df_sub_position.subject_end, df_sub_position.subject_type, sub):
    replace_sub.append(sen[:i]+f'@ {k} {word} @'+sen[j:])
    
replace_sub = pd.Series(replace_sub)

In [177]:
replace_sub

0                                                                                                                                                                                                                                                                                                                            지난 15일 @ ORG MBC @ '탐사기획 스트레이트'가 이 사실을 보도했다.
1                                                                                                                                                                                                                                                               사랑스러운 ‘@ PER 프린세스 프링 @’의 이름은 봄의 공주님: Princess(s)Pring이란 뜻으로 탄생, 부활, 청춘 등 여러 가지 의미를 담아 생일왕국의 공주님의 이름이 되었다.
2       한편, 본인(이근안)을 모델로 한 MBC 특집드라마 가 1995년 6.25 특집극으로 편성될 예정이었는데 제작비 문제 때문에 연출자 고석만 PD(당시 단막극 팀장)가 MBC에 지원을 요청했지만 거절당한 데다 MBC가 @ ORG 경찰 @과의 관계 약화를 염려하여 제작을 달갑지 않게 여겼고 이 과정에서 연출자가 1995년 3월 29일 MBC에 사표를 제출한 후 프리랜서를 선언하여 편성이 무산됐으며 고석만 PD는 의 제작과 관련한 MBC와의 마찰 외에도 1994년 10월 "평프로듀

In [178]:
# object_entity  f'# {type} {entity} #' 형태로 표현 

replace_sub_obj = []

for sen, i, j, k, word in zip(replace_sub, df_obj_position.object_start, df_obj_position.object_end, df_obj_position.object_type, obj):
    if sen[i:j]==word:
        replace_sub_obj.append(sen[:i]+f'# {k} {word} #'+sen[j:])
    else:
        replace_sub_obj.append(sen[:i+8]+f'# {k} {word} #'+sen[j+8:])
    
replace_sub_obj = pd.Series(replace_sub_obj)

In [179]:
df_label = df.label

In [180]:
df_label

0       100
1       100
2       100
3       100
4       100
       ... 
7760    100
7761    100
7762    100
7763    100
7764    100
Name: label, Length: 7765, dtype: int64

In [181]:
df_replace_sub_obj = pd.DataFrame({'id':df.index, 'sentence':replace_sub_obj, 'label':df_label})

In [182]:
df_replace_sub_obj

,id,sentence,label
0,0,지난 15일 @ ORG MBC @ '# ORG 탐사기획 스트레이트 #'가 이 사실을 보도했다.,100
1,1,"사랑스러운 ‘@ PER 프린세스 프링 @’의 이름은 봄의 공주님: Princess(s)Pring이란 뜻으로 탄생, 부활, 청춘 등 여러 가지 의미를 담아 생일왕국의 # POH 공주 #님의 이름이 되었다.",100
2,2,"한편, 본인(이근안)을 모델로 한 MBC 특집드라마 가 # DAT 1995년 # 6.25 특집극으로 편성될 예정이었는데 제작비 문제 때문에 연출자 고석만 PD(당시 단막극 팀장)가 MBC에 지원을 요청했지만 거절당한 데다 MBC가 @ ORG 경찰 @과의 관계 약화를 염려하여 제작을 달갑지 않게 여겼고 이 과정에서 연출자가 1995년 3월 29일 MBC에 사표를 제출한 후 프리랜서를 선언하여 편성이 무산됐으며 고석만 PD는 의 제작과 관련한 MBC와의 마찰 외에도 1994년 10월 ""평프로듀서로 일하고 싶다""며 당시 맡고 있던 단막극 책임PD(종합병원 전원일기 등등) 보직 사퇴의사를 밝혔지만 MBC 측의 반대로 무산되자 프리랜서를 선언했다.",100
3,3,"# PER 정창손 #은 김질과 같이 대궐로 달려가 고변하며 '신은 실로 모르고 김질만 혼자 참여하였는데, 김질의 죄는 만번 죽어 마땅합니다.'라고 하니, @ PER 세조 @가 특별히 김질을 사면하였다.",100
4,4,"당시 @ ORG 민주당 @ 이진련 시의원은 # PER 권영진 # 시장에게 ""긴급 생계자금을 왜 현금으로 지원하지 않느냐""고 따졌다.",100
...,...,...,...
7760,7760,코로나19 방역 조치의 일환으로 국민의 움직임을 통제하려는 @ ORG 정부 @의 시도를 # LOC 이탈리아 # 국민이 어느 정도까지 수용할지도 의문이다.,100
7761,7761,선 연구원은 “위식도역류질환치료제인 케이캡이 92억원 판매되면서 2019년 연간 328억원 매출을 기록했다”라며 “올해 약 500억원 이상의 매출 달성은 충분히 가능해 보인다. 기존 품목의 고른 성장과 신규 품목의 고성장세에 힘입어 올해 @ ORG 종근당 @의 탑라인은 # DAT 전년 #대비 8% 이상 high single digit 성장률을 기록할 수 있을 것으로 보인다”라고 언급했다.,100
7762,7762,"@ ORG 한국전기안전공사 @(사장 # PER 조성완 #)는 8월 1일부로, 3급 간부직원에 대한 승진·이동 인사를 아래와 같이 발령하였다.",100
7763,7763,# DAT 1987년 # @ PER B. 슈나이더 @(B. Schneider)에 의해 만들어졌다.,100


In [60]:
df_replace_sub_obj.to_csv('mark_and_type_test.csv')

### train data

In [215]:
df = pd.read_csv('../dataset/train/train.csv')

In [216]:
# subject_entity index 와 타입을 받아서 DataFrame 에 저장

subject_start = []
subject_end = []
subject_type = []

for i in df['subject_entity']:
    i = int(i[1:-1].split("'start_idx': ")[1].split(',')[0])

    subject_start.append(i)

for j in df['subject_entity']:
    j = int(j[1:-1].split("'end_idx': ")[1].split(',')[0])+1

    subject_end.append(j)
    
for k in df['subject_entity']:
    k = str(k[1:-1].split("'type': ")[1][1:4])

    subject_type.append(k)
    
subject_start = pd.Series(subject_start)
subject_end = pd.Series(subject_end)
subject_type = pd.Series(subject_type)

df_sub_position = pd.DataFrame({'subject_start':subject_start, 'subject_end':subject_end, 'subject_type':subject_type})

In [217]:
df_sub_position

,subject_start,subject_end,subject_type
0,24,27,ORG
1,19,24,ORG
2,21,25,ORG
3,13,18,ORG
4,22,31,ORG
...,...,...,...
32465,93,96,PER
32466,7,10,PER
32467,0,3,ORG
32468,6,10,ORG


In [218]:
# subject_entity 단어 확인

sub = []

for sen, i, j in zip(df.sentence, df_sub_position.subject_start, df_sub_position.subject_end):
    sub.append(sen[i:j])

In [219]:
sub = pd.Series(sub)

In [220]:
sub

0              비틀즈
1            민주평화당
2             광주FC
3            아성다이소
4        요미우리 자이언츠
           ...    
32465          유기준
32466          최시형
32467          완도군
32468         JTBC
32469          화순군
Length: 32470, dtype: object

In [221]:
# object_entity index 와 타입을 받아서 DataFrame 에 저장

object_start = []
object_end = []
object_type = []

for i in df['object_entity']:
    i = int(i[1:-1].split("'start_idx': ")[1].split(',')[0])

    object_start.append(i)

for j in df['object_entity']:
    j = int(j[1:-1].split("'end_idx': ")[1].split(',')[0])+1

    object_end.append(j)
    
for k in df['object_entity']:
    k = str(k[1:-1].split("'type': ")[1][1:4])

    object_type.append(k)
    
object_start = pd.Series(object_start)
object_end = pd.Series(object_end)
object_type = pd.Series(object_type)

df_obj_position = pd.DataFrame({'object_start':object_start, 'object_end':object_end, 'object_type':object_type})

In [222]:
# subject_entity 단어 확인

obj = []

for sen, i, j in zip(df.sentence, df_obj_position.object_start, df_obj_position.object_end):
    obj.append(sen[i:j])

In [223]:
obj = pd.Series(obj)

In [224]:
obj

0          조지 해리슨
1            대안신당
2        한국프로축구연맹
3             박정부
4            1967
           ...   
32465    부산 서구·동구
32466         손병희
32467         신우철
32468       중앙홀딩스
32469         구충곤
Length: 32470, dtype: object

In [225]:
# subject_entity  @ {type} {entity} @ 형태로 표현 

replace_sub = []

for sen, i, j, k, word in zip(df.sentence, df_sub_position.subject_start, df_sub_position.subject_end, df_sub_position.subject_type, sub):
    replace_sub.append(sen[:i]+f'@ {k} {word} @'+sen[j:])
    
replace_sub = pd.Series(replace_sub)

In [226]:
replace_sub

0                                                                                                                                                                  〈Something〉는 조지 해리슨이 쓰고 @ ORG 비틀즈 @가 1969년 앨범 《Abbey Road》에 담은 노래다.
1                                                                                                                                                                       호남이 기반인 바른미래당·대안신당·@ ORG 민주평화당 @이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
2                                                                                                                          K리그2에서 성적 1위를 달리고 있는 @ ORG 광주FC @는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
3                                                                                                                                               균일가 생활용품점 (주)@ ORG 아성다이소 @(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
4                                                                           

In [227]:
# object_entity  f'# {type} {entity} #' 형태로 표현 

replace_sub_obj = []

for sen, i, j, k, word in zip(replace_sub, df_obj_position.object_start, df_obj_position.object_end, df_obj_position.object_type, obj):
    if sen[i:j]==word:
        replace_sub_obj.append(sen[:i]+f'# {k} {word} #'+sen[j:])
    else:
        replace_sub_obj.append(sen[:i+8]+f'# {k} {word} #'+sen[j+8:])
    
replace_sub_obj = pd.Series(replace_sub_obj)

In [228]:
df_label = df.label

In [229]:
df_label

0                      no_relation
1                      no_relation
2                    org:member_of
3        org:top_members/employees
4                      no_relation
                   ...            
32465              per:employee_of
32466               per:colleagues
32467    org:top_members/employees
32468                  no_relation
32469    org:top_members/employees
Name: label, Length: 32470, dtype: object

In [230]:
df_replace_sub_obj = pd.DataFrame({'id':df.index, 'sentence':replace_sub_obj, 'label':df_label})

In [231]:
df_replace_sub_obj

,id,sentence,label
0,0,〈Something〉는 # PER 조지 해리슨 #이 쓰고 @ ORG 비틀즈 @가 1969년 앨범 《Abbey Road》에 담은 노래다.,no_relation
1,1,호남이 기반인 바른미래당·# ORG 대안신당 #·@ ORG 민주평화당 @이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.,no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 @ ORG 광주FC @는 지난 26일 # ORG 한국프로축구연맹 #으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.,org:member_of
3,3,균일가 생활용품점 (주)@ ORG 아성다이소 @(대표 # PER 박정부 #)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.,org:top_members/employees
4,4,# DAT 1967 #년 프로 야구 드래프트 1순위로 @ ORG 요미우리 자이언츠 @에게 입단하면서 등번호는 8번으로 배정되었다.,no_relation
...,...,...,...
32465,32465,"한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등록 마감 결과, 강석호(3선·경북 영양·영덕·봉화·울진)-이장우(재선·대전 동구), @ PER 유기준 @(4선·# LOC 부산 서구·동구 #)-박성중(초선·서울 서초을), 김선동(재선·서울 도봉을)-김종석(초선·비례), 심재철(5선·경기 안양시동안구을)-김재원(3선·경북 상주·군위·의성·청송) 등 4개 조가 등록했다고 밝혔다.",per:employee_of
32466,32466,"법포는 다시 @ PER 최시형 @, 서병학, # PER 손병희 # 직계인 북접과 다시 서장옥, 전봉준, 김개남을 지도자로 하는 남접으로 나뉘게 된다.",per:colleagues
32467,32467,@ ORG 완도군 @(군수 # PER 신우철 #)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 229개 기초지자체 중 최상위 등급인 A등급을 받으면서 전국에서 가장 높은 교통안전 인식 수준을 갖고 있는 것으로 나타났다.,org:top_members/employees
32468,32468,"중앙일보, @ ORG JTBC @ 회장을 지낸 이후 # ORG 중앙홀딩스 # 회장, 재단법인 한반도평화만들기 이사장, 한국기원 총재를 맡고 있다.",no_relation


In [232]:
from sklearn.model_selection import train_test_split

data = df_replace_sub_obj
target = df_replace_sub_obj['label']

# train_test_split
mark_and_type_train, mark_and_type_valid = train_test_split(data, test_size=0.2, shuffle=True, stratify=target, random_state=2023)

In [233]:
mark_and_type_train.to_csv('mark_and_type_train.csv')
mark_and_type_valid.to_csv('mark_and_type_dev.csv')

## UNK > 3 인 문장 drop

In [234]:
# 베이스라인 코드의 load_data.py 코드 긁어와서 사용함

def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = []
  object_entity = []
  for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
    i = i[1:-1].split(',')[0].split(':')[1]
    j = j[1:-1].split(',')[0].split(':')[1]

    subject_entity.append(i)
    object_entity.append(j)
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
  return out_dataset

def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)
  
  return dataset

def tokenized_dataset(dataset, tokenizer):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)
  tokenized_sentences = tokenizer(
      concat_entity,
      list(dataset['sentence']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=256,
      add_special_tokens=True,
      )
  return tokenized_sentences

In [235]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer

MODEL_NAME = "klue/roberta-large" 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [236]:
train_dataset = load_data("../dataset/train/train.csv")
tokenized_train = tokenized_dataset(train_dataset, tokenizer)

### 토크나이징이 잘 됬는지 확인해 봄

In [237]:
print(*tokenized_train.input_ids[:3])

tensor([    0,    11, 29830,    11,     2,    11,  8373, 14113,  2234,    11,
            2,   168, 30985, 14451,  7088,  4586,   169,   793,  8373, 14113,
         2234,  2052,  1363,  2088, 29830,  2116, 14879,  2440,  6711,   170,
        21406, 26713,  2076, 25145,  5749,   171,  1421,   818,  2073,  4388,
         2062,    18,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [238]:
len(tokenized_train.input_ids)

32470

In [239]:
print(*tokenized_train)

input_ids token_type_ids attention_mask


In [240]:
print(tokenized_train.token_type_ids[0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])


In [241]:
tokenized_train.attention_mask[0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

### UNK 토큰 개수를 통해 문장 추출하기 위해 pandas DataFrame 으로 변환

In [242]:
input_ids_df = pd.DataFrame(tokenized_train.input_ids.numpy())

In [243]:
input_ids_np = tokenized_train.input_ids.numpy()

In [244]:
input_ids_pd = pd.DataFrame(input_ids_np)

In [245]:
input_ids_pd # 확인

,0,1,2,3,4,5,6,7,8,9,...,231,232,233,234,235,236,237,238,239,240
0,0,11,29830,11,2,11,8373,14113,2234,11,...,1,1,1,1,1,1,1,1,1,1
1,0,11,3772,2139,2267,2481,11,2,11,5605,...,1,1,1,1,1,1,1,1,1,1
2,0,11,4104,10904,11,2,11,3629,17287,20212,...,1,1,1,1,1,1,1,1,1,1
3,0,11,27930,24393,2024,11,2,11,6580,2144,...,1,1,1,1,1,1,1,1,1,1
4,0,11,20289,20562,11,2,11,14925,11,2,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32465,0,11,5967,2456,11,2,11,3902,5905,100,...,1,1,1,1,1,1,1,1,1,1
32466,0,11,28246,2444,11,2,11,1284,2394,2364,...,1,1,1,1,1,1,1,1,1,1
32467,0,11,18272,2242,11,2,11,31102,2313,11,...,1,1,1,1,1,1,1,1,1,1
32468,0,11,15710,11,2,11,4097,2827,2500,2255,...,1,1,1,1,1,1,1,1,1,1


In [246]:
((input_ids_pd==3).sum(axis=1)!=0)

0        False
1        False
2        False
3        False
4        False
         ...  
32465    False
32466    False
32467    False
32468    False
32469    False
Length: 32470, dtype: bool

In [247]:
((input_ids_pd==3).sum(axis=1)!=0).sum(axis=0)

3218

### UNK 토큰 개수별 문장 수 확인

In [248]:
# train_dataset = load_data("../dataset/train/train.csv")

In [249]:
((input_ids_pd==3).sum(axis=1)).value_counts()

0     29252
1      1256
2      1066
3       480
4       189
5       110
6        58
8        28
7        22
10        4
12        2
9         2
11        1
dtype: int64

### UNK 토큰이 많은 문장 출력해서 확인

In [250]:
UNK_over_n = 9

condition = ((input_ids_pd==3).sum(axis=1)>UNK_over_n)

sen_id = iter(train_dataset.index[condition])
UNK_count = iter(((input_ids_pd==1).sum(axis=1))[condition])
UNK_sentence = iter(train_dataset.sentence[condition])
UNK_sub = iter(train_dataset.subject_entity[condition])
UNK_obj = iter(train_dataset.object_entity[condition])
UNK_ids = iter(tokenized_train.input_ids[condition])

In [251]:
print('sentence id :', next(sen_id))
print('UNK count :', next(UNK_count))
print(next(UNK_sub))
print(next(UNK_obj))
print(next(UNK_sentence))
print(next(UNK_ids))

sentence id : 4597
UNK count : 157
 '쥘 아르망'
 '프랑스'
프랑스군이 투언안 요새를 함락하면서 프랑스 치안청장인 프랑수아 쥘 아르망(François Jules Harmand) 이 베트남 협판대학사 쩐딘뚝(Trần Đình Túc), 이부상서 응우옌쫑헙(Nguyễn Trọng Hợp)과 조약을 체결했기 때문에 종종 ‘아르망 조약’으로도 불린다.
tensor([    0,    11,     3,  5996,  2087,    11,     2,    11,  4510,    11,
            2,  4510,  2242,  2052,  1801,  2347,  2283,  9730,  2138, 20547,
         5643,  2112,  4510, 11409,  2270,  2121,  2179, 23664,     3,  5996,
         2087,    12,     3,    46, 30480,  2041, 17015,  9811,  2039,    13,
         1504,  5713,  1922,  2025,  2104,  2218,  2063,  1607,  2695,  3124,
           12,     3,     3,     3,    13,    16, 27754,  2290,  2112,     3,
           12,     3,     3,     3,    13,   604,  8604,  2069,  5186,  2371,
         2015,  3624,  2170,  7286,   114,  5996,  2087,  8604,   115,  3603,
         2119, 12732,    18,     2,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
         

### UNK 토큰 (3) 3개 보다 많은 행 train_dataset 에서 drop

In [252]:
train_droped = train_dataset.drop(labels=train_dataset.index[(input_ids_pd==3).sum(axis=1)>3], axis=0)

In [253]:
len(train_droped)

32054

In [254]:
from sklearn.model_selection import train_test_split

data = train_droped
target = train_droped['label']

# train_test_split
drop_unk3_train, drop_unk3_valid = train_test_split(data, test_size=0.2, shuffle=True, stratify=target, random_state=2023)

In [255]:
drop_unk3_train.to_csv('drop_unk3_train.csv')
drop_unk3_valid.to_csv('drop_unk3_dev.csv')

## \@ type entity \@ 에서 UNK > 3 인 행 삭제한 데이터 생성

In [258]:
mark_type_drop = df_replace_sub_obj.drop(labels=df.index[(input_ids_pd==3).sum(axis=1)>3], axis=0)

In [261]:
len(mark_type_drop)

32054

In [263]:
mark_type_drop.head(5)

,id,sentence,label
0,0,〈Something〉는 # PER 조지 해리슨 #이 쓰고 @ ORG 비틀즈 @가 1969년 앨범 《Abbey Road》에 담은 노래다.,no_relation
1,1,호남이 기반인 바른미래당·# ORG 대안신당 #·@ ORG 민주평화당 @이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.,no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 @ ORG 광주FC @는 지난 26일 # ORG 한국프로축구연맹 #으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.,org:member_of
3,3,균일가 생활용품점 (주)@ ORG 아성다이소 @(대표 # PER 박정부 #)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.,org:top_members/employees
4,4,# DAT 1967 #년 프로 야구 드래프트 1순위로 @ ORG 요미우리 자이언츠 @에게 입단하면서 등번호는 8번으로 배정되었다.,no_relation


In [264]:
data = mark_type_drop
target = mark_type_drop['label']

# train_test_split
mark_type_drop_train, mark_type_drop_valid = train_test_split(data, test_size=0.2, shuffle=True, stratify=target, random_state=2023)

mark_type_drop_train.to_csv('mark_type_drop_train.csv')
mark_type_drop_valid.to_csv('mark_type_drop_dev.csv')